## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config_api import gmaps_key as gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Evanston,US,2021-07-24 21:36:30,42.0411,-87.6901,91.62,70,40,1.01,thunderstorm
1,1,Punta Arenas,CL,2021-07-24 21:36:31,-53.1500,-70.9167,41.11,75,75,11.50,broken clouds
2,2,Chaoyang,CN,2021-07-24 21:36:31,41.5703,120.4586,74.28,87,59,4.61,broken clouds
3,3,Saint-Philippe,RE,2021-07-24 21:36:31,-21.3585,55.7679,67.59,79,37,4.97,light rain
4,4,Albany,US,2021-07-24 21:36:31,42.6001,-73.9662,81.25,52,87,3.00,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Enter minimum temperature for trip'))
max_temp = float(input('Enter maximum temperature for trip'))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

custom_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                    (city_data_df["Max Temp"] >= min_temp)]
custom_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
1,1,Punta Arenas,CL,2021-07-24 21:36:31,-53.1500,-70.9167,41.11,75,75,11.50,broken clouds
2,2,Chaoyang,CN,2021-07-24 21:36:31,41.5703,120.4586,74.28,87,59,4.61,broken clouds
3,3,Saint-Philippe,RE,2021-07-24 21:36:31,-21.3585,55.7679,67.59,79,37,4.97,light rain
4,4,Albany,US,2021-07-24 21:36:31,42.6001,-73.9662,81.25,52,87,3.00,overcast clouds
5,5,Rikitea,PF,2021-07-24 21:36:32,-23.1203,-134.9692,69.26,54,0,1.43,clear sky
6,6,Torbay,CA,2021-07-24 21:36:32,47.6666,-52.7314,66.97,79,75,13.80,broken clouds
7,7,Bluff,NZ,2021-07-24 21:36:32,-46.6000,168.3333,44.80,69,100,10.07,overcast clouds
8,8,Kapaa,US,2021-07-24 21:36:32,22.0752,-159.3190,82.72,79,75,4.00,light rain
9,9,Hermanus,ZA,2021-07-24 21:36:32,-34.4187,19.2345,51.91,56,14,5.68,few clouds
10,10,Hay River,CA,2021-07-24 21:36:33,60.8156,-115.7999,53.58,66,75,11.50,broken clouds


In [9]:
# 4a.Determine if there are any empty rows.
custom_cities_df.count()

City_ID            644
City               644
Country            635
Date               644
Lat                644
Lng                644
Max Temp           644
Humidity           644
Cloudiness         644
Wind Speed         644
Current Weather    644
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = custom_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Lat", "Lng",'Current Weather']].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
1,Punta Arenas,CL,41.11,-53.1500,-70.9167,broken clouds,
2,Chaoyang,CN,74.28,41.5703,120.4586,broken clouds,
3,Saint-Philippe,RE,67.59,-21.3585,55.7679,light rain,
4,Albany,US,81.25,42.6001,-73.9662,overcast clouds,
5,Rikitea,PF,69.26,-23.1203,-134.9692,clear sky,
6,Torbay,CA,66.97,47.6666,-52.7314,broken clouds,
7,Bluff,NZ,44.80,-46.6000,168.3333,overcast clouds,
8,Kapaa,US,82.72,22.0752,-159.3190,light rain,
9,Hermanus,ZA,51.91,-34.4187,19.2345,few clouds,
10,Hay River,CA,53.58,60.8156,-115.7999,broken clouds,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel")


        

Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot fin

In [13]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
1,Punta Arenas,CL,41.11,-53.1500,-70.9167,broken clouds,Hotel Dreams Del Estrecho
2,Chaoyang,CN,74.28,41.5703,120.4586,broken clouds,Chaoyang Genting International Hotel
3,Saint-Philippe,RE,67.59,-21.3585,55.7679,light rain,"Chambres d'hôte ""La Trinité"""
4,Albany,US,81.25,42.6001,-73.9662,overcast clouds,
5,Rikitea,PF,69.26,-23.1203,-134.9692,clear sky,People ThankYou
...,...,...,...,...,...,...,...
685,Banjarmasin,ID,69.89,-3.3244,114.5910,few clouds,Swiss-Belhotel Borneo Banjarmasin
686,Jilotepec,MX,72.34,19.6000,-96.9333,broken clouds,Auto-Hotel Mediterráneo
687,Pula,HR,76.06,44.8683,13.8481,clear sky,Park Plaza Histria Pula
689,Banda Aceh,ID,80.37,5.5577,95.3222,overcast clouds,Sulthan Hotel International


In [14]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
1,Punta Arenas,CL,41.11,-53.1500,-70.9167,broken clouds,Hotel Dreams Del Estrecho
2,Chaoyang,CN,74.28,41.5703,120.4586,broken clouds,Chaoyang Genting International Hotel
3,Saint-Philippe,RE,67.59,-21.3585,55.7679,light rain,"Chambres d'hôte ""La Trinité"""
5,Rikitea,PF,69.26,-23.1203,-134.9692,clear sky,People ThankYou
6,Torbay,CA,66.97,47.6666,-52.7314,broken clouds,Gallows Cove
...,...,...,...,...,...,...,...
685,Banjarmasin,ID,69.89,-3.3244,114.5910,few clouds,Swiss-Belhotel Borneo Banjarmasin
686,Jilotepec,MX,72.34,19.6000,-96.9333,broken clouds,Auto-Hotel Mediterráneo
687,Pula,HR,76.06,44.8683,13.8481,clear sky,Park Plaza Histria Pula
689,Banda Aceh,ID,80.37,5.5577,95.3222,overcast clouds,Sulthan Hotel International


In [16]:
# 8a. Create the output File (CSV)
output_data_file= "../weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd> 
<dt> </dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>  
</dl>
"""


#Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = (info_box_template.format(**row) for index, row in hotel_df.iterrows())

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add a marker layer for each city to the map. 

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig = gmaps.figure(center=(38.02077, 23.73524), zoom_level=1.8)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig




Figure(layout=FigureLayout(height='420px'))